# 1. Introduction 

In [ ]:
from tqdm import tqdm
import numpy as np

## 1. Import Azure ML Python Python SDK

In [2]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.83


## 2. Authentication and initializing Azure Machine Learning Workspace

In [4]:
from azureml.core import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: myworkspace
Azure region: westeurope
Subscription id: 558bd446-4212-46a2-908c-9ab0a628705e
Resource group: azure-ml-service-rg


## 3. Create an experiment

In [ ]:
experiment = Experiment(workspace=ws, name='introduction')

## 4. Starting the run

In [ ]:
# start logging for the run
run = experiment.start_logging()

# access the run id for use later
run_id = run.id

# change the scale factor on different runs to see how you can compare multiple runs
scale_factor = 2

# change the category on different runs to see how to organize data in reports
category = 'Red'

Once a run is started you can see the run in the portal by simply typing ``run``.  Clicking on the "Link to Portal" link will take you to the Run Details page that shows the metrics you have logged and other run properties.  You can refresh this page after each logging statement to see the updated results.

In [ ]:
run

You can also view an experiement similarly by typing `experiment`.  The portal link will take you to the experiment's Run History page that shows all runs and allows you to analyze trends across multiple runs.

In [ ]:
experiment

## 5. Log metrics

Metrics are visible in the run details page in the AzureML portal and also can be analyzed in experiment reports.  The run details page looks as below and contains tabs for Details, Outputs, Logs, and Snapshot.  
* The Details page displays attributes about the run, plus logged metrics and images.  Metrics that are vectors appear as charts.  
* The Outputs page contains any files, such as models, you uploaded into the "outputs" directory from your run into storage.  If you place files in the "outputs" directory locally, the files are automatically uploaded on your behald when the run is completed.
* The Logs page allows you to view any log files created by your run.  Logging runs created in notebooks typically do not generate log files.
* The Snapshot page contains a snapshot of the directory specified in the ''start_logging'' statement, plus the notebook at the time of the ''start_logging'' call.  This snapshot and notebook can be downloaded from the Run Details page to continue or reproduce an experiment.

The following cell logs a string metric.  A string metric is simply a string value associated with a name.  A string metric String metrics are useful for labelling runs and to organize your data.  Typically you should log all string parameters as metrics for later analysis - even information such as paths can help to understand how individual experiements perform differently.

String metrics can be used in the following ways:
* Plot in hitograms
* Group by indicators for numerical plots
* Filtering runs

String metrics appear in the **Tracked Metrics** section of the Run Details page and can be added as a column in Run History reports.

In [ ]:
# log a string metric
run.log(name='Category', value=category)

In [ ]:
# log numerical values
run.log(name="scale factor", value = scale_factor)
run.log(name='Magic Number', value=42 * scale_factor)

In [ ]:
fibonacci_values = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
scaled_values = (i * scale_factor for i in fibonacci_values)

# Log a list of values. Note this will generate a single-variable line chart.
run.log_list(name='Fibonacci', value=scaled_values)

for i in tqdm(range(-10, 10)):
    # log a metric value repeatedly, this will generate a single-variable line chart.
    run.log(name='Sigmoid', value=1 / (1 + np.exp(-i)))

In [ ]:
# create a dictionary to hold a table of values
sines = {}
sines['angle'] = []
sines['sine'] = []

for i in tqdm(range(-10, 10)):
    angle = i / 2.0 * scale_factor
    
    # log a 2 (or more) values as a metric repeatedly. This will generate a 2-variable line chart if you have 2 numerical columns.
    run.log_row(name='Cosine Wave', angle=angle, cos=np.cos(angle))
        
    sines['angle'].append(angle)
    sines['sine'].append(np.sin(angle))

# log a dictionary as a table, this will generate a 2-variable chart if you have 2 numerical columns
run.log_table(name='Sine Wave', value=sines)

In [ ]:
file_name = 'outputs/myfile.txt'

with open(file_name, "w") as f:
    f.write('This is an output file that will be uploaded.\n')

# Upload the file explicitly into artifacts 
run.upload_file(name = file_name, path_or_stream = file_name)

## 6. Complete the run

Calling `run.complete()` marks the run as completed and triggers the output file collection.  If for any reason you need to indicate the run failed or simply need to cancel the run you can call `run.fail()` or `run.cancel()`.

In [ ]:
run.complete()

## 7. Run a project

In [ ]:
TRAIN_FOLDER_NAME = 'train'
TRAIN_FILE_NAME = 'train.py'

In [ ]:
import os
os.makedirs(os.path.join(".", TRAIN_FOLDER_NAME), exist_ok=True)

In [ ]:
%%writefile $TRAIN_FOLDER_NAME/$TRAIN_FILE_NAME

from azureml.core import Run

submitted_run = Run.get_context()
submitted_run.log(name="message", value="Hello from run!")


In [ ]:
est = Estimator(source_directory=TRAIN_FOLDER_NAME, 
                entry_script=TRAIN_FILE_NAME,
                compute_target='local')

## 7. Analyze results

You can refresh the run in the Azure portal to see all of your results.  In many cases you will want to analyze runs that were performed previously to inspect the contents or compare results.  Runs can be fetched from their parent Experiment object using the ``Run()`` constructor or the ``experiment.get_runs()`` method. 

In [ ]:
fetched_run = Run(experiment, run_id)
fetched_run

Call ``run.get_metrics()`` to retrieve all the metrics from a run.

In [ ]:
fetched_run.get_metrics()

See the files uploaded for this run by calling ``run.get_file_names()``

In [ ]:
fetched_run.get_file_names()

Once you know the file names in a run, you can download the files using the ``run.download_file()`` method

In [ ]:
import os
os.makedirs('files', exist_ok=True)

for f in run.get_file_names():
    dest = os.path.join('files', f.split('/')[-1])
    print('Downloading file {} to {}...'.format(f, dest))
    fetched_run.download_file(f, dest)   

Often when you analyze the results of a run, you may need to tag that run with important personal or external information.  You can add a tag to a run using the ``run.tag()`` method.  AzureML supports valueless and valued tags.

In [ ]:
fetched_run.tag("My Favorite Run")
fetched_run.tag("Competition Rank", 1)

fetched_run.get_tags()